# 【課題】腫瘍の良性／悪性を予測（分類）しよう

## データを読み込む

`breast_cancer_wisconsin_data.csv` を読み込みましょう。

In [2]:
# Pandasを使ってbreast_cancer_wisconsin_data.csvを読み込む（変更しないでください）
import pandas as pd
bc_data = pd.read_csv("breast_cancer_wisconsin_data.csv")

読み込んだデータについて確認します。

In [3]:
# 読み込んだデータが何行何列かをshapeで確認する（変更しないでください）
print(bc_data.shape)

(569, 33)


In [4]:
# 読み込んだデータの先頭5行を確認する（変更しないでください）
bc_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
# 読み込んだデータの先頭の1行だけを確認する（変更しないでください）
bc_data.iloc[0, :]

id                           842302
diagnosis                         M
radius_mean                   17.99
texture_mean                  10.38
perimeter_mean                122.8
area_mean                    1001.0
smoothness_mean              0.1184
compactness_mean             0.2776
concavity_mean               0.3001
concave points_mean          0.1471
symmetry_mean                0.2419
fractal_dimension_mean      0.07871
radius_se                     1.095
texture_se                   0.9053
perimeter_se                  8.589
area_se                       153.4
smoothness_se              0.006399
compactness_se              0.04904
concavity_se                0.05373
concave points_se           0.01587
symmetry_se                 0.03003
fractal_dimension_se       0.006193
radius_worst                  25.38
texture_worst                 17.33
perimeter_worst               184.6
area_worst                   2019.0
smoothness_worst             0.1622
compactness_worst           

このCSVファイルには30以上の列があります。主要な列のみ、以下に概要を記載します。

- id：連番
- diagnosis："B"か"M"の文字が格納されている（"B"：良性、"M"：悪性）
- radius_mean：中心から外周までの平均距離（半径）
- texture_mean：グレースケール（色の濃さ）の平均値
- perimeter_mean：外周の平均の長さ

今回は `radius_mean`（半径）と `perimeter_mean`（外周の長さ）を説明変数、`diagnosis`（良性か悪性か）を目的変数として分類を行ないます。

データの前処理や分割を行なう前に、スライスを使って、`radius_mean`列と`perimeter_mean`列を変数`X`、`diagnosis` 列のみを変数 `y` に格納してください。また、PandasでCSVを読み込んだため、データはDataFrame形式となっています。そのため、データはndarray形式に変換してください。

In [7]:
# スライスとndarrayへの変換をして X と y を作成する
X = bc_data[['radius_mean', 'perimeter_mean']].values
y = bc_data['diagnosis'].values

# データの形状と型を確認します
print(f'Xの形状: {X.shape}')
print(f'yの形状: {y.shape}')
print(f'Xの型: {type(X)}')
print(f'yの型: {type(y)}')

# 最初の5行を表示して中身を確認します
print('\n--- X (最初の5行) ---')
print(X[:5])
print('\n--- y (最初の5行) ---')
print(y[:5])


Xの形状: (569, 2)
yの形状: (569,)
Xの型: <class 'numpy.ndarray'>
yの型: <class 'numpy.ndarray'>

--- X (最初の5行) ---
[[ 17.99 122.8 ]
 [ 20.57 132.9 ]
 [ 19.69 130.  ]
 [ 11.42  77.58]
 [ 20.29 135.1 ]]

--- y (最初の5行) ---
['M' 'M' 'M' 'M' 'M']


## データの前処理を行なう

さて、`y` には "B" もしくは "M" という文字データが入っています。数値化した方がコンピュータは学習しやすくなるので、データの前処理の1つ「カテゴリ値の数値化」を実行しましょう。カテゴリ値の数値化をするには `LabelEncoder` というものを利用します。

[sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

※ `LabelEncoder` は、レッスン本文では登場しませんでしたので、以下のコードをそのまま実行すれば大丈夫です。このコードは、上記の公式ドキュメントの内容に沿って記述されています。**なお、このとき `y` は、NumPyのndarray形式に変換されている必要があります。**

In [8]:
# カテゴリ値の数値化
# （変更しないでください）
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["B", "M"])    # 良性：0, 悪性：1
y = le.transform(y)

# 数値化した状態を確認してみる（変更しないでください）
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

## データを訓練データとテストデータに分ける

`X` および `y` を訓練データとテストデータに分けましょう。その際、訓練データ8割、テストデータ2割としてください。また、並び順がランダムになるよう、分割してください。

In [11]:
# 訓練データ8割、テストデータ2割に分割する
# 訓練データとテストデータに分割するためのインポート
from sklearn.model_selection import train_test_split

# 訓練データ＝train、テストデータ＝test
# test_size=0.2 とすることで、テストデータの割合を20%に指定
# shuffle=True を指定して、データを分割する前にシャッフル（ランダムに並び替え）します
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# 分割後のデータの形状を確認します
print("X_trainの形状:", X_train.shape)
print("X_testの形状:", X_test.shape)
print("y_trainの形状:", y_train.shape)
print("y_testの形状:", y_test.shape)


X_trainの形状: (455, 2)
X_testの形状: (114, 2)
y_trainの形状: (455,)
y_testの形状: (114,)


## モデルを作成して訓練する

ここでは、レッスン本文でも利用した「サポートベクターマシン」（`SVC`）の線形分類モデルを使います。また、作成したモデルに訓練データを渡して、学習を行ないます。

In [12]:
# SVCの線形分類モデルを作成する
# サポートベクターマシン（SVC）をインポートします
from sklearn.svm import SVC

# kernel='linear' を指定して、線形カーネルを使用します
# random_state=0 を指定して、結果の再現性を確保します
model = SVC(kernel='linear', random_state=0)

# 訓練データを使って訓練を行なう
# model.fit() に訓練データ（X_train, y_train）を渡します
model.fit(X_train, y_train)

# モデルの訓練が完了したことを確認するために、モデルの情報を表示します
print("モデルの訓練が完了しました。")
print("使用したモデル:", model)


モデルの訓練が完了しました。
使用したモデル: SVC(kernel='linear', random_state=0)


## 期待する性能が出たかを評価する

分類を行ないましょう。

In [13]:
# テストデータを使って分類を実行する
# model.predict() にテストデータ（X_test）を渡して、予測結果を得ます
y_pred = model.predict(X_test)

print("テストデータによる予測が完了しました。")

テストデータによる予測が完了しました。


この分類結果を用いて、モデルの評価を行ないます。

まずは、予測値と実際の値をそのまま表示してください。

In [14]:
# 予測値を表示する
# 上で作成した予測結果（y_pred）の中身を確認します
print("--- 予測値 ---")
print(y_pred)

--- 予測値 ---
[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1
 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [15]:
# 実際の値を表示する
# 予測値（y_pred）と比較するために、実際の正解データ（y_test）を表示します
print("--- 実際の値 ---")
print(y_test)

--- 実際の値 ---
[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0
 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1
 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0
 1 0 1]


予測の精度を混同行列で示しましょう。

In [16]:
# 混同行列で集計結果を表示する
# 混同行列を計算・表示するためのインポート
from sklearn.metrics import confusion_matrix

# confusion_matrixに実際の値（y_test）と予測値（y_pred）を渡します
cm = confusion_matrix(y_test, y_pred)

print("--- 混同行列 ---")
print(cm)

--- 混同行列 ---
[[68  1]
 [ 7 38]]


最後に `classification_report()` を実行し、結果を表示してください。

In [17]:
# classification_report()の実行結果を表示する
# classification_reportを計算・表示するためのインポート
from sklearn.metrics import classification_report

# classification_reportに実際の値（y_test）と予測値（y_pred）を渡します
report = classification_report(y_test, y_pred)

print("--- 分類レポート ---")
print(report)


--- 分類レポート ---
              precision    recall  f1-score   support

           0       0.91      0.99      0.94        69
           1       0.97      0.84      0.90        45

    accuracy                           0.93       114
   macro avg       0.94      0.91      0.92       114
weighted avg       0.93      0.93      0.93       114

